In [1]:
%matplotlib inline

import dateutil
import pandas as pd
import requests

pd.options.display.float_format = '{:.2f}'.format

In [2]:
BITCOIN_ABUSE_URL = 'https://www.bitcoinabuse.com/api/reports/distinct'

REPORT = []

NEXT_URL = None

while True:
    if not NEXT_URL:
        resp = requests.get(BITCOIN_ABUSE_URL)
    else:
        resp = requests.get(NEXT_URL)

    if not resp.ok:
        continue

    data = resp.json()

    REPORT += data['data']

    if not data['next_page_url']:
        break

    NEXT_URL = data['next_page_url']

In [3]:
df = pd.DataFrame(REPORT)

df['reported_at'].apply(dateutil.parser.parse)

df.sort_values('count', ascending=False).head()

,address,count,reported_at
40,1EZS92K4xJbymDLwG4F7PNF5idPE62e9XY,421,2018-10-25 15:43:38
45,1MN7A7QqQaAVoxV4zdjdrnEHXmjhzcQ4Bq,281,2018-10-25 14:53:36
7,1JTtwbvmM7ymByxPYCByVYCwasjH49J3Vj,276,2018-10-25 23:12:37
43,1NXNt72qfMhPZDffUEqryCYpEUzyR6LmgH,272,2018-10-25 15:19:46
160,19D67Tgb3neJiTHd8pZDEBYmUn2qSjxEeB,173,2018-10-23 11:59:45


In [4]:
df.set_index('address').to_csv('output/bitcoin-abuse-reports.csv')

In [5]:
def get_total_received_and_sent(row):
    resp = requests.get(f'https://blockchain.info/rawaddr/{row["address"]}')

    if not resp.ok:
        return row

    data = resp.json()

    row['total_received'] = data['total_received']
    row['total_sent'] = data['total_sent']
    row['final_balance'] = data['final_balance']

    return row


df.head(1).apply(lambda row: get_total_received_and_sent(row), axis=1).head()

,address,count,reported_at,total_received,total_sent,final_balance
0,1Abx4ytuJ2Mb5HtuCpbUuUuJ1q8cdrXXYT,6,2018-10-25 23:59:47,0,0,0


In [6]:
df = df.apply(lambda row: get_total_received_and_sent(row), axis=1)

df.head()

,address,count,final_balance,reported_at,total_received,total_sent
0,1Abx4ytuJ2Mb5HtuCpbUuUuJ1q8cdrXXYT,6,0.00,2018-10-25 23:59:47,0.00,0.00
1,1MTDXjt14YLEBDNxZ8WEq23FNa1QfHXMbc,1,0.00,2018-10-25 23:51:00,0.00,0.00
2,1JMSH4oDSuGteB46G7Yg1FLQXeqfVPJyyU,1,0.00,2018-10-25 23:50:27,0.00,0.00
3,1LZQGS99RUCvQvT5Qce7LkrWtMtcXPSdWZ,110,46882183.00,2018-10-25 23:44:54,46882183.00,0.00
4,1Le9CMak1WFS9ERpfmRkvHKTdugkobDqCe,1,13743600.00,2018-10-25 23:34:17,13743600.00,0.00


In [13]:
df['total_received'] = df['total_received'].apply(lambda x: x / 100000000)
df['total_sent'] = df['total_sent'].apply(lambda x: x / 100000000)
df['final_balance'] = df['final_balance'].apply(lambda x: x / 100000000)

df.head()

,address,count,final_balance,reported_at,total_received,total_sent,total_received_cad,total_sent_cad
0,1Abx4ytuJ2Mb5HtuCpbUuUuJ1q8cdrXXYT,6,0.00,2018-10-25 23:59:47,0.00,0.00,0.00,0.00
1,1MTDXjt14YLEBDNxZ8WEq23FNa1QfHXMbc,1,0.00,2018-10-25 23:51:00,0.00,0.00,0.00,0.00
2,1JMSH4oDSuGteB46G7Yg1FLQXeqfVPJyyU,1,0.00,2018-10-25 23:50:27,0.00,0.00,0.00,0.00
3,1LZQGS99RUCvQvT5Qce7LkrWtMtcXPSdWZ,110,0.47,2018-10-25 23:44:54,0.00,0.00,3942.34,0.00
4,1Le9CMak1WFS9ERpfmRkvHKTdugkobDqCe,1,0.14,2018-10-25 23:34:17,0.00,0.00,1155.70,0.00


In [14]:
df.sort_values('total_received', ascending=False).head(10)

,address,count,final_balance,reported_at,total_received,total_sent,total_received_cad,total_sent_cad
458,1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s,2,18744.25,2018-10-16 05:34:52,0.04,0.04,29963003479.46,29805382446.43
2091,32zsaL5r7YERsqyC7EJbgrbL1ehFQdSt8K,1,0.00,2018-04-15 07:31:12,0.00,0.00,473875179.27,473875179.27
2090,1H4Ng3NkDLiEhEPJ4EVUvc7Da13R3PYafq,1,0.00,2018-04-15 07:33:00,0.00,0.00,185442136.81,185442133.13
1496,17up7EiuJze4UTggQwQZAvvQG87cGqJ9U9,1,0.00,2018-09-17 18:16:22,0.00,0.00,22948178.24,22948178.24
1707,1JsACYBoRCYkz7DSgyKurMyibbmHwcHbPd,1,0.00,2018-08-15 08:49:20,0.00,0.00,5959072.97,5959072.97
171,3CwdHnqNTWjryE2qnyVa82Gcz2LwKdkGHh,1,0.34,2018-10-23 06:38:12,0.00,0.00,2997459.55,2994624.15
343,1BEFmdLRKQ7VsARX7uHtNKSVPW4sze9CcU,1,10.46,2018-10-18 13:26:34,0.00,0.00,1997187.60,1909203.46
790,15SW4gXvkn8hfog398iTxyuCkM6TdfpFhp,1,30.95,2018-10-04 18:06:39,0.00,0.00,1776540.09,1516271.77
2070,13AkbXoqjWaH4eSMaWkB6RVmFa7vXfKj4y,2,0.00,2018-05-08 16:01:16,0.00,0.00,954761.82,954761.82
414,3QnKobHyVWMU1v2c75oH6ztuiUGehNp3rH,1,73.62,2018-10-16 17:59:17,0.00,0.00,619042.23,0.00


In [15]:
resp = requests.get('https://api.coindesk.com/v1/bpi/currentprice/CAD.json')

rate = resp.json()['bpi']['CAD']['rate_float']

rate

8409.0349

In [18]:
df['total_received_cad'] = df['total_received'].apply(lambda x: round(x * rate, 2))
df['total_sent_cad'] = df['total_sent'].apply(lambda x: round(x * rate, 2))
df['final_balance'] = df['final_balance'].apply(lambda x: round(x * rate, 2))

df.head()

,address,count,final_balance,reported_at,total_received,total_sent,total_received_cad,total_sent_cad
0,1Abx4ytuJ2Mb5HtuCpbUuUuJ1q8cdrXXYT,6,0.00,2018-10-25 23:59:47,0.00,0.00,0.00,0.00
1,1MTDXjt14YLEBDNxZ8WEq23FNa1QfHXMbc,1,0.00,2018-10-25 23:51:00,0.00,0.00,0.00,0.00
2,1JMSH4oDSuGteB46G7Yg1FLQXeqfVPJyyU,1,0.00,2018-10-25 23:50:27,0.00,0.00,0.00,0.00
3,1LZQGS99RUCvQvT5Qce7LkrWtMtcXPSdWZ,110,3942.34,2018-10-25 23:44:54,0.00,0.00,0.00,0.00
4,1Le9CMak1WFS9ERpfmRkvHKTdugkobDqCe,1,1155.70,2018-10-25 23:34:17,0.00,0.00,0.00,0.00


In [19]:
df.sort_values('total_received', ascending=False).head(10)

,address,count,final_balance,reported_at,total_received,total_sent,total_received_cad,total_sent_cad
458,1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s,2,157621033.03,2018-10-16 05:34:52,0.04,0.04,299.63,298.05
2091,32zsaL5r7YERsqyC7EJbgrbL1ehFQdSt8K,1,0.00,2018-04-15 07:31:12,0.00,0.00,4.74,4.74
2090,1H4Ng3NkDLiEhEPJ4EVUvc7Da13R3PYafq,1,3.67,2018-04-15 07:33:00,0.00,0.00,1.85,1.85
1496,17up7EiuJze4UTggQwQZAvvQG87cGqJ9U9,1,0.00,2018-09-17 18:16:22,0.00,0.00,0.23,0.23
1707,1JsACYBoRCYkz7DSgyKurMyibbmHwcHbPd,1,0.00,2018-08-15 08:49:20,0.00,0.00,0.06,0.06
171,3CwdHnqNTWjryE2qnyVa82Gcz2LwKdkGHh,1,2835.40,2018-10-23 06:38:12,0.00,0.00,0.03,0.03
343,1BEFmdLRKQ7VsARX7uHtNKSVPW4sze9CcU,1,87984.14,2018-10-18 13:26:34,0.00,0.00,0.02,0.02
790,15SW4gXvkn8hfog398iTxyuCkM6TdfpFhp,1,260268.32,2018-10-04 18:06:39,0.00,0.00,0.02,0.02
2070,13AkbXoqjWaH4eSMaWkB6RVmFa7vXfKj4y,2,0.00,2018-05-08 16:01:16,0.00,0.00,0.01,0.01
414,3QnKobHyVWMU1v2c75oH6ztuiUGehNp3rH,1,619042.23,2018-10-16 17:59:17,0.00,0.00,0.01,0.00


In [20]:
df[df['total_sent'] == 0]

,address,count,final_balance,reported_at,total_received,total_sent,total_received_cad,total_sent_cad
0,1Abx4ytuJ2Mb5HtuCpbUuUuJ1q8cdrXXYT,6,0.00,2018-10-25 23:59:47,0.00,0.00,0.00,0.00
1,1MTDXjt14YLEBDNxZ8WEq23FNa1QfHXMbc,1,0.00,2018-10-25 23:51:00,0.00,0.00,0.00,0.00
2,1JMSH4oDSuGteB46G7Yg1FLQXeqfVPJyyU,1,0.00,2018-10-25 23:50:27,0.00,0.00,0.00,0.00
3,1LZQGS99RUCvQvT5Qce7LkrWtMtcXPSdWZ,110,3942.34,2018-10-25 23:44:54,0.00,0.00,0.00,0.00
4,1Le9CMak1WFS9ERpfmRkvHKTdugkobDqCe,1,1155.70,2018-10-25 23:34:17,0.00,0.00,0.00,0.00
7,1JTtwbvmM7ymByxPYCByVYCwasjH49J3Vj,276,33327.03,2018-10-25 23:12:37,0.00,0.00,0.00,0.00
8,1PfbxCJkGNTZC7yFtHHhtPnZyiwQEUqAmu,1,0.00,2018-10-25 23:08:10,0.00,0.00,0.00,0.00
9,1CKUjp9VNWFd1ysDbsq27dK9icG3vjeGr6,12,0.00,2018-10-25 23:01:54,0.00,0.00,0.00,0.00
10,167jvgbxeM5o2BWwgzsSdspVXUtcaH58PT,2,0.00,2018-10-25 23:01:27,0.00,0.00,0.00,0.00
11,1Fb8btYEng9DXYg3GY5hYSXAaLvGvWXRe2,2,0.00,2018-10-25 22:32:56,0.00,0.00,0.00,0.00
